In [1]:
import pandas as pd

In [3]:
df = pd.read_table("OFFSET 0828 1220.OFS",sep=r"\n",header=None, engine='python', names=['header'])

In [4]:
def convert_in2mm(x):
    return x*25.4

In [5]:
df_tools = df[df['header'].str.contains('P\d+R')].copy()

df_tools['Tool'] = df_tools['header'].str.extract(r'P(\d+)R').astype('int')
df_tools['Category'] = df_tools['header'].str.extract(r'(L\d+)P').astype('str')
df_tools['Value'] = df_tools['header'].str.extract(r"P\d+R(-?[0-9]\d*\.\d+)").astype('float').apply(convert_in2mm)

cat_dict = {
    "L10":"Tool Length Geo",
    "L11":"Tool Length Wear",
    "L12":"Tool Radius Geo",
    "L13":"Tool Radius Wear",
            }

df_tools['Category'] = df_tools['Category'].map(cat_dict)

In [6]:
df_works = df[df['header'].str.contains('P\d+X')].copy()

df_works['Tool'] = df_works['header'].str.extract(r'P(\d+)X').astype('int')
df_works['Category'] = df_works['header'].str.extract(r'(L\d+)P').astype('str')
df_works['Tool-Cat'] = df_works['Tool'].astype('str') + "-" + df_works['Category']
df_works['X_Value'] = df_works['header'].str.extract(r"X(-?[0-9]\d*\.\d+)").astype('float').apply(convert_in2mm)
df_works['Y_Value'] = df_works['header'].str.extract(r"Y(-?[0-9]\d*\.\d+)").astype('float').apply(convert_in2mm)
df_works['Z_Value'] = df_works['header'].str.extract(r"Z(-?[0-9]\d*\.\d+)").astype('float').apply(convert_in2mm)
df_works['W_Value'] = df_works['header'].str.extract(r"W(-?[0-9]\d*\.\d+)").astype('float')
df_works['A_Value'] = df_works['header'].str.extract(r"A(-?[0-9]\d*\.\d+)").astype('float')

coord_dict = {
    "0-L2" : "COMMON",
    "1-L2" : "G54",
    "2-L2" : "G55",
    "3-L2" : "G56",
    "4-L2" : "G57",
    "1-L20" : "G54.1P1",
    "2-L20" : "G54.1P2",
    "3-L20" : "G54.1P3",
    "4-L20" : "G54.1P4"
}

df_works['Coord'] = df_works["Tool-Cat"].map(coord_dict)

In [7]:
df_tools_pivot = df_tools.pivot_table(index="Tool", columns="Category", values="Value")
df_tools_pivot

Category,Tool Length Geo,Tool Length Wear,Tool Radius Geo,Tool Radius Wear
Tool,,,,
1,187.236862,0.0,-0.088138,0.0
2,181.865016,0.0,-0.011176,0.0
3,183.240172,0.0,-0.008636,0.0
4,179.789328,0.0,-0.002794,0.0
5,180.806598,0.0,-0.006096,0.0
...,...,...,...,...
396,0.000000,0.0,0.000000,0.0
397,0.000000,0.0,0.000000,0.0
398,0.000000,0.0,0.000000,0.0


In [8]:
df_tools_pivot.to_clipboard()

In [9]:
df_works_pivot = df_works[['Coord','X_Value', 'Y_Value','Z_Value', 'W_Value', 'A_Value']].dropna(subset='Coord')
df_works_pivot = df_works_pivot.set_index('Coord')
df_works_pivot

,X_Value,Y_Value,Z_Value,W_Value,A_Value
Coord,,,,,
COMMON,0.000000,0.000000,0.000000,0.0,0.0
G54,-487.386376,-279.442164,-580.294242,0.0,0.0
G55,-283.604716,-279.383998,-580.289670,0.0,0.0
G56,-487.372152,-279.395428,-440.074812,20.0,-90.0
G57,-283.605732,-279.364694,-440.041538,-90.0,-90.0
G54.1P1,-487.386376,-279.442164,-439.959242,20.0,0.0
G54.1P2,-283.604716,-279.383998,-439.954670,20.0,0.0
G54.1P3,-487.386376,-279.399746,-440.082432,20.0,-90.0
G54.1P4,-283.604716,-279.388316,-440.043316,-90.0,-90.0


In [10]:
df_works_pivot.to_clipboard()